<a href="https://colab.research.google.com/github/federicasibilla/ScenicOrNot/blob/main/Preparazione_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scenic Or Not

## Dataset di immagini dal gioco https://scenicornot.datasciencelab.co.uk/

Il gioco propone immagini di scene outdoor scattate in Inghilterra, Scozia e Galles. Gli utenti possono assegnare ad ogni scena un punteggio intero da 1 a 10. Il dataset generato è composto da 7 colonne:


*   ID della fotografia
*   Latitudine
*   Longitudine
*   Punteggio medio
*   Varianza 
*   Elenco dei voti
*   URL per la pagina di http://www.geograph.org.uk/ contenente l'immagine

In questo notebook si implementa la creazione degli array utili allo studio del problema di riconoscere la scenicità di un luogo outdoor:



*   Array contenente gli array RGB delle immagini che sono state votate più di 10 volte
*   Array delle corrispondenti distribuzioni dei voti ricevuti per ciascuna immagine

Riportiamo un esempio di riga del dataset:

---

*39	 53.9849  	-1.41281  	3.6000  	1.8400	  5,4,4,3,1,6,3,2,4,4	 http://www.geograph.org.uk/photo/324*

---
negli array saranno contenuti, rispettivamente, l'immagine (riscalata a 224x224 pixel) contenuta nella pagina del link e l'array [0.1, 0.1, 0.2, 0.4, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0].






In [1]:
# import

import numpy as np
import tensorflow as tf
import pandas as pd

from google.colab import files
from google.colab import drive
import joblib

from collections import Counter

from PIL import Image
import requests
from bs4 import BeautifulSoup
from io import BytesIO

In [ ]:
# connessione. aldrive per salvare gli array finiti

drive.mount('/content/drive')

In [ ]:
# creazione del dizionario dalla lettura del file

data = []
with open('/content/votes.tsv', 'r') as file:
    next(file)
    for line in file:
        values = line.strip().split()
        url = values[-1]
        integers = [int(x) for x in values[5].split(',')]
        row_dict = {'URL': url, 'Votes': integers}
        data.append(row_dict)

In [ ]:
# trasformazione in un dizionario con proporzione di conteggi per punteggio e selezione delle immagini con almeno 11 voti

conteggi_totali = Counter()
data_dict={}

for dizionario in data:
    url = dizionario['URL']
    votes = dizionario['Votes']
    conteggi = Counter(votes)
    data_dict[url] = [conteggi.get(i, 0) for i in range(1, 11)]
    somma = sum(data_dict[url])
    if somma < 10:
      del data_dict[url]
    else:
      data_dict[url] = [valore / somma for valore in data_dict[url]]

In [ ]:
# visuazlizzazione dizionario

count = 0
for chiave, valore in data_dict.items():
    print(chiave, valore)
    count += 1
    if count == 3:
        break

In [ ]:
# funzioni per la conversione degli URL in array RGB ridimensionati

#  funzione che dall' URL di una pagina estrae quello di un'immagine 

def get_image_url_from_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Trova l'elemento HTML che contiene l'URL dell'immagine
    image_element = soup.find('img')
    if image_element:
        image_url = image_element['src']
        return image_url
    else:
        return None

#  funzione che converte da URL a matrice RGB

def url_to_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = img.convert('RGB')
    img_array = np.array(img)
    return img_array


# funzione per il ridimensionamento delle immagini

def resize_img(array_img, width, height):
    img_resized = tf.image.resize(array_img, [width, height])
    return img_resized

In [ ]:
# creare array di array con le immagini, ridimensionate

images = []
votes = []

for url, values in data_dict.items():

    web_page_url = url
    image_url = get_image_url_from_page(web_page_url)

    if image_url:
      img_array = url_to_image(image_url)
      # Utilizza l'immagine come desiderato
    else:
      print("URL dell'immagine non trovato.")

    images.append(resize_img(img_array, 224,224))
    votes.append(values)

In [ ]:
#salvataggio array

np.save('/content/drive/MyDrive/array_immagini224_completo11voti.npy', images)
np.save('/content/drive/MyDrive/array_voti_completo11voti.npy', votes)